In [ ]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

In [ ]:
from f1tenth_gym_jax import make
from f1tenth_gym_jax.envs.utils import Param
import jax
import numpy as np

In [ ]:
num_agents = 5
num_envs = 100

In [ ]:
env = make(f"Spielberg_{num_agents}_noscan_time_v0")

In [ ]:
rng = jax.random.PRNGKey(0)
rng, _rng = jax.random.split(rng)
reset_rng = jax.random.split(_rng, num_envs)

In [ ]:
obsv, env_state = jax.vmap(env.reset)(reset_rng)

In [ ]:
rng, _rng = jax.random.split(rng)
action_rng = jax.random.split(_rng, num_envs*num_agents).reshape((num_agents, num_envs, -1))
actions = {k: jax.vmap(env.action_space(k).sample)(action_rng[i]) for i, k in enumerate(env.agents)}

In [ ]:
rng, _rng = jax.random.split(rng)
step_rng = jax.random.split(_rng, num_envs)
obsv, env_state, _, _, info = jax.vmap(env.step)(step_rng, env_state, actions)

In [ ]:
def env_init(rng):
    rng, _rng = jax.random.split(rng)
    reset_rng = jax.random.split(_rng, num_envs)
    obsv, env_state = jax.vmap(env.reset)(reset_rng)
    return (env_state, obsv, rng)

def env_step(runner_state, unused):
    env_state, last_obs, rng = runner_state

    rng, _rng = jax.random.split(rng)
    action_rng = jax.random.split(_rng, num_envs*num_agents).reshape((num_agents, num_envs, -1))
    actions = {k: jax.vmap(env.action_space(k).sample)(action_rng[i]) for i, k in enumerate(env.agents)}
    for k in env.agents:
        actions[k] = actions[k].at[:, 0].set(0.0)
        actions[k] = actions[k].at[:, 1].set(2.0)

    rng, _rng = jax.random.split(rng)
    step_rng = jax.random.split(_rng, num_envs)
    obsv, env_state, _, _, info = jax.vmap(env.step)(step_rng, env_state, actions)

    runner_state = (env_state, obsv, rng)
    return runner_state, runner_state

In [ ]:
runner_state = env_init(rng)
last_runner_state, all_runner_states = jax.lax.scan(env_step, runner_state, length=2000)

In [ ]:
all_runner_states[0].cartesian_states.shape

In [ ]:
assert False

In [ ]:
import importlib
# import f1tenth_gym_jax
import f1tenth_gym_jax.envs.rendering.renderer as r
importlib.reload(r)

In [ ]:
player = r.TrajRenderer(env)

In [ ]:
try:
    player.render(np.array(all_runner_states[0].cartesian_states))
except Exception as e:
    print(e)
    player.close()